# Lab 11: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

2022-04-26 01:56:11,508 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = BaseLocalization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-04-26 01:56:26,849 | INFO     |:  | Number of observations per grid cell: 18
2022-04-26 01:56:26,851 | INFO     |:  | Precaching Views...


/Users/roninsharma/Desktop/Cornell/ECE 5960/Fast-Robots/Lab11/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2022-04-26 01:56:33,257 | INFO     |:  | Precaching Time: 6.403 secs
2022-04-26 01:56:33,258 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-04-26 01:56:33,258 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


In [3]:
# Initialize the Trajectory object
traj = Trajectory(loc)

In [4]:
# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Print the latest observation data stored in the member variable obs_range_data
print(loc.obs_range_data)

[[0.71951102]
 [0.78792011]
 [2.09482577]
 [1.57051175]
 [1.41726235]
 [1.33916654]
 [1.47424957]
 [0.96886687]
 [0.78024917]
 [1.6884828 ]
 [1.80478309]
 [2.11948374]
 [1.60862645]
 [0.76911938]
 [1.38518276]
 [1.60735144]
 [2.24752034]
 [2.08939699]]


In [5]:
# In the docstring, "pose" refers to a numpy array with elements (x,y,yaw) in (meters, meters, degrees)

def compute_control(curr_pose, prev_pose):
    """ Given the current and previous odometry poses, this function extracts
    the control information based on the odometry motion model.

    Args:
        cur_pose  ([Pose]): Current Pose
        prev_pose ([Pose]): Previous Pose 

    Returns:
        [delta_rot_1]: Rotation 1  (degrees)
        [delta_trans]: Translation (meters)
        [delta_rot_2]: Rotation 2  (degrees)
    """
    
    delta_x = curr_pose[0] - prev_pose[0]
    delta_y = curr_pose[1] - prev_pose[1]
    delta_thetha = curr_pose[2] - prev_pose[2]
    
    delta_rot_1 = np.degrees( np.arctan2(delta_y, delta_x) - prev_pose[2] )
    delta_rot_2 = np.degrees( delta_thetha - delta_rot_1 )
    
    delta_trans = ( ( delta_x ** 2 ) + ( delta_y ** 2 ) ) ** 0.5

    return mapper.normalize_angle(delta_rot_1), delta_trans, mapper.normalize_angle(delta_rot_2)

def odom_motion_model(cur_pose, prev_pose, u):
    """ Odometry Motion Model

    Args:
        cur_pose  ([Pose]): Current Pose
        prev_pose ([Pose]): Previous Pose
        u (rot1, trans, rot2) (float, float, float): A tuple with control data in the format 
                                                   format (rot1, trans, rot2) with units (degrees, meters, degrees)


    Returns:
        prob [float]: Probability p(x'|x, u)
    """
    
    delta_rot_1, delta_trans, delta_rot_2 = compute_control(cur_pose, prev_pose)
    
    gaussian_rot1 = loc.gaussian(u[0], delta_rot_1, loc.odom_rot_sigma)
    gaussian_rot2 = loc.gaussian(u[2], delta_rot_2, loc.odom_rot_sigma)
    
    gaussian_trans = loc.gaussian(u[1], delta_trans, loc.odom_trans_sigma)
    
    return gaussian_rot1 * gaussian_rot2 * gaussian_trans

def prediction_step(cur_odom, prev_odom):
    """ Prediction step of the Bayes Filter.
    Update the probabilities in loc.bel_bar based on loc.bel from the previous time step and the odometry motion model.

    Args:
        cur_odom  ([Pose]): Current Pose
        prev_odom ([Pose]): Previous Pose
    """
    # previous state: bel_bar
    # current state: bel
    
    maxX = mapper.MAX_CELLS_X
    maxY = mapper.MAX_CELLS_Y
    maxAngle = mapper.MAX_CELLS_A
    
    u = compute_control(cur_odom, prev_odom)
    
    # previous dimensions
    for r in range(maxX):
        for s in range(maxY):
            for t in range(maxAngle):
                
                # skip over small probabilities
                if (loc.bel[r][s][t] > 0.0001):
                
                    prep = mapper.from_map(r, s, t)

                    # current dimensions
                    for c in range(maxX):
                        for d in range(maxY):
                            for e in range(maxAngle):

                                    curp = mapper.from_map(c, d, e)
                                    prob = odom_motion_model(curp, prep, u)
                                    belp = loc.bel[r][s][t]
                                    loc.bel_bar[c][d][e] += prob * belp

    # Special thanks to the lab handout
    loc.bel_bar = loc.bel_bar / np.sum(loc.bel_bar)

def sensor_model(obs):
    """ This is the equivalent of p(z|x).


    Args:
        obs ([ndarray]): A 1D array consisting of the true observations for a specific robot pose in the map 

    Returns:
        float: product of all probabilities (of the likelihoods of each individual sensor measurement)
    """

    prob = 1
    
    for i in range(len(obs)):
        prob *= loc.gaussian(loc.obs_range_data[i][0], obs[i], loc.sensor_sigma)
    
    return prob

def update_step():
    """ Update step of the Bayes Filter.
    Update the probabilities in loc.bel based on loc.bel_bar and the sensor model.
    """
    
    maxX = mapper.MAX_CELLS_X
    maxY = mapper.MAX_CELLS_Y
    maxAngle = mapper.MAX_CELLS_A
    
    # current dimensions
    for c in range(maxX):
        for d in range(maxY):
            for e in range(maxAngle):

                loc.bel[c][d][e] = loc.bel_bar[c][d][e] * sensor_model(mapper.get_views(c, d, e))
    
    # Special thanks to the lab handout
    loc.bel = loc.bel / np.sum(loc.bel)

# Run the Bayes Filter
The cells below utilizes the functions declared above to run each iteration of the Bayes filter algorithm to localize the robot in the grid. <br>

In each iteration of the loop:
- Execute robot motion (get $u_{t}$ as previous and current odom) 
- Perform prediction step (calculate $\overline{bel}$)
- Print information regarding Prediction step
- Execute robot rotation behavior to get observation data (get $z_{t}$)
- Perform update step (calculate $bel$)
- Print information regarding update step


**<ins>NOTE</ins>**: 
- During initial testing, you may want to limit the iteration to only the first time step (i.e t=0) instead of looping through the entire trajectory.
- <span style="color:rgb(0,150,0)">If you make changes to any of the functions above, make sure to re-run the above cells before executing the cell below.</span>
- The functions *print_prediction_stats()* and *print_update_stats()* are helper functions defined in <a href="../localization.py">localization.py</a> and may be changed to suit your needs.
- <span style="color:rgb(0,150,0)">Always run an initial update step before the first prediction step.</span>

#### The cell below contains code to initialize a uniform probability distribution and perform the update step of the Bayes Filter to localize the robot.

In [6]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2022-04-26 01:57:22,852 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-04-26 01:57:22,852 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2022-04-26 01:57:26,060 | INFO     |: ---------- UPDATE STATS -----------
2022-04-26 01:57:26,073 | INFO     |: GT index      : (6, 4, 9)
2022-04-26 01:57:26,074 | INFO     |: Bel index     : (5, 3, 9) with prob = 0.5106828
2022-04-26 01:57:26,075 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-04-26 01:57:26,078 | INFO     |: GT            : (0.000, 0.000, 360.000)
2022-04-26 01:57:26,081 | INFO     |: Belief        : (0.000, -0.305, 10.000)
2022-04-26 01:57:26,082 | INFO     |: POS ERROR     : (-0.000, 0.305, 350.000)
2022-04-26 01:57:26,084 | INFO     |: ---------- UPDATE STATS -----------


In [7]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2022-04-26 01:57:52,846 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-26 01:57:52,858 | INFO     |: GT index         : (6, 3, 7)
2022-04-26 01:57:52,859 | INFO     |: Prior Bel index  : (5, 2, 2) with prob = 0.0560032
2022-04-26 01:57:52,860 | INFO     |: POS ERROR        : (0.287, 0.521, 90.275)
2022-04-26 01:57:52,864 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-26 01:57:56,000 | INFO     |: ---------- UPDATE STATS -----------
2022-04-26 01:57:56,007 | INFO     |: GT index      : (6, 3, 7)
2022-04-26 01:57:56,009 | INFO     |: Bel index     : (6, 4, 6) with prob = 1.0
2022-04-26 01:57:56,010 | INFO     |: Bel_bar prob at index = 8.129339509106195e-05
2022-04-26 01:57:56,014 | INFO     |: GT            : (0.287, -0.089, 320.275)
2022-04-26 01:57:56,017 | INFO     |: Belief        : (0.305, 0.000, -50.000)
2022-04-26 01:57:56,019 | INFO     |: POS ERROR     : (-0.018, -0.089, 370.275)
2022-04-26 01:57:56,024 | INF

2022-04-26 01:58:34,017 | INFO     |: POS ERROR     : (-0.138, -0.162, 2511.180)
2022-04-26 01:58:34,028 | INFO     |: ---------- UPDATE STATS -----------
-------------------------------------


----------------- 7 -----------------
2022-04-26 01:58:36,832 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-26 01:58:36,841 | INFO     |: GT index         : (11, 3, 13)
2022-04-26 01:58:36,842 | INFO     |: Prior Bel index  : (11, 1, 11) with prob = 0.0920525
2022-04-26 01:58:36,843 | INFO     |: POS ERROR        : (-0.082, 0.809, 2557.006)
2022-04-26 01:58:36,847 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-26 01:58:39,965 | INFO     |: ---------- UPDATE STATS -----------
2022-04-26 01:58:39,973 | INFO     |: GT index      : (11, 3, 13)
2022-04-26 01:58:39,974 | INFO     |: Bel index     : (11, 4, 13) with prob = 1.0
2022-04-26 01:58:39,976 | INFO     |: Bel_bar prob at index = 2.7577536268420777e-09
2022-04-26 01:58:39,981 | INFO     |: GT            : (1.747,

2022-04-26 01:59:22,799 | INFO     |: Bel_bar prob at index = 1.7681147813240345e-07
2022-04-26 01:59:22,800 | INFO     |: GT            : (0.009, -0.142, 5274.168)
2022-04-26 01:59:22,805 | INFO     |: Belief        : (0.000, -0.305, -130.000)
2022-04-26 01:59:22,814 | INFO     |: POS ERROR     : (0.009, 0.163, 5404.168)
2022-04-26 01:59:22,818 | INFO     |: ---------- UPDATE STATS -----------
-------------------------------------


----------------- 14 -----------------
2022-04-26 01:59:27,088 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-26 01:59:27,101 | INFO     |: GT index         : (4, 2, 1)
2022-04-26 01:59:27,102 | INFO     |: Prior Bel index  : (3, 2, 2) with prob = 0.0809194
2022-04-26 01:59:27,103 | INFO     |: POS ERROR        : (0.253, 0.289, 5381.722)
2022-04-26 01:59:27,106 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-26 01:59:30,400 | INFO     |: ---------- UPDATE STATS -----------
2022-04-26 01:59:30,411 | INFO     |: GT index      : (